In [ ]:
import argparse
import glob
import os
import random
from importlib.resources import path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile

# from genericpath import isfile
from openslide import OpenSlide

from omegaconf import OmegaConf
# Load config
preproc_conf = OmegaConf.load("../conf/preproc.yaml")
preproc_conf = preproc_conf['classic_mil_on_embeddings_bag']['bracs_224_224_patches']

In [ ]:
parent_folder = preproc_conf.data_root_dir
parent_folder

In [ ]:
preproc_conf.img_dir_lvl4

In [ ]:
def get_slide_file_path(slide_id):
    #train
    data_dir = parent_folder
    # holdout
    #data_dir = '<YOUR_PATH>/'
    #slide_fp = os.path.join(data_dir,'*', '*', '*', f'{slide_id}.svs')
    slide_fp = os.path.join(data_dir, f'{slide_id}.svs')    
    return glob.glob(slide_fp)[0]


class PatchGeneratorPixel:
    def __init__(self, slide_path, slide_level) -> None:
        self.slide_path = slide_path
        self.slide_dir_path = os.path.dirname(os.path.realpath(self.slide_path))
        self.slide_name = os.path.splitext(os.path.basename(self.slide_path))[0]
        self.slide_level = slide_level
        self.patch_dir_path = preproc_conf.img_dir_lvl4 # 'bracs_save_level4_npy/'
        os.makedirs(self.patch_dir_path, exist_ok=True)
        
        #os.path.join(self.slide_dir_path, "..", f"patch_by_pixel_level_{self.slide_level}_resized_all")

    def get_slide_name_w_ext(self):
        return os.path.basename(self.slide_path)
    
    def read_lvl4_then_write(self):
        slide_openslide = OpenSlide(self.slide_path)
        img_openslide = slide_openslide.read_region((0, 0), 2, slide_openslide.level_dimensions[2])
        img_openslide_RGB = img_openslide.convert("RGB")
        img_openslide_np = np.array(img_openslide_RGB)
        print(img_openslide_np.shape)

        save_slide_folder = os.path.join(self.patch_dir_path, self.slide_name)
        print(save_slide_folder)
        np.save( save_slide_folder+'_level'+str(self.slide_level+2)+'.npy', img_openslide_np ) 

In [ ]:
get_slide_file_path('BRACS_1356')

In [ ]:
get_slide_file_path('BRACS_1855')

In [ ]:
get_slide_file_path('BRACS_1335')

In [ ]:
slide_openslide = OpenSlide('<YOUR_PATH>/BRACS_WSI/BRACS_1335.svs')

In [ ]:
slide_openslide.level_dimensions

In [ ]:
slide_openslide.level_downsamples

#### -> levels are: 0, 2, 4, 5

### Process

In [ ]:
slide_level = 2 # this is 4 for the competition dataset
    
# ALL OF THE DATA
data_dir = parent_folder

slides_fp = os.path.join(data_dir, '*')
slides_list = [ j.split('/')[-1].split('.svs')[0] for j in sorted(glob.glob(slides_fp)) ]

len(slides_list) # number of WSIs 

In [ ]:
slides_list[:5]

In [ ]:
nr_of_slides_processed = 0

for slide_path in slides_list:
    try:
        print('slide path:', slide_path)
        patch_generator = PatchGeneratorPixel( slide_path=get_slide_file_path(slide_path), 
                                               slide_level=slide_level)
        patch_generator.read_lvl4_then_write()
        print('processed slides:', nr_of_slides_processed)
        nr_of_slides_processed += 1
    except:
        print('passed') # this means no processing if current slide does not have this resolution level
        pass